In [23]:
### This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib 
from matplotlib import pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
"""
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
"""
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

import FlowCal
from keras.utils import Sequence
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from sklearn import preprocessing

In [7]:
# Sort the file names
np.random.seed(1)
FCS_data = []
labels = []
data_dir = "SamusikData/"
for i in os.listdir(data_dir):
    if "csv" in i and "assignments" in i and not "04" in i:
        labels.append(data_dir + i)
    elif "fcs" in i and not "04" in i:
        FCS_data.append(data_dir + i)

FCS_data.sort()
labels.sort()
#print(FCS_data)
#print(labels)

In [38]:
# Keras DataGenerator
# We use this to load the data as we need it every epoch

class DataGenerator(Sequence):
    
    """
    fcs_files = List of .fcs files in directory
    labels = list of .csv files in directory (sorted the same as fcs_files)
    batch_size = number of scRNA data points to consider in a batch
       DataGenerator will sample [batch_size/len(list_IDs)] samples from each fcs file
       Each file has ~87K samples
    n_classes = 25 (24 cells, 1 no cell type listed (0))    
    
    """
    def __init__(self, fcs_files, labels, batch_size=30000, n_classes=25, shuffle=True):
        self.n_features = 51 # Each sample has 51 features
        self.batch_size = batch_size
        self.labels = labels
        self.fcs_files = fcs_files
        self.n_classes = n_classes
        self.shuffle = shuffle
        
        # Make an index over all samples
        # self.length is an int for the total number of samples
        # self.indices is a list of ints that tell us which index belongs to what file
        self.length, self.indices = self.compute_length(labels)
        self.indexes = np.array([i for i in range(self.length)])
        self.on_epoch_end()
        
    def compute_length(self, labels):
        L = 0
        FL = []
        for l in labels:
            df = pd.read_csv(l)
            L = L + len(df)
            FL.append(L)
        return L, FL
            
    def __len__(self):
        return self.length // self.batch_size
    
    def __getitem__(self, index):
        
        X = np.zeros((self.batch_size, self.n_features))
        #y = np.zeros(self.batch_size)
        
        # start and end index X and y
        start = 0
        end = 0
        
        # Prev max indexes globally for all samples
        prev_max = 0
        
        # idx/index both hold the global sample indexes we want
        idx = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        
        # bin them according to files in which they reside
        for i, max_file_index in enumerate(self.indices):
            
            file_idx = idx[idx < max_file_index] - prev_max
            
            if len(file_idx) == 0:
                continue
                
            end = start + len(file_idx)
            #print(len(file_idx))
            
            # Read each fcs file to get the samples
            si = FlowCal.io.FCSData(self.fcs_files[i])
            si = np.arcsinh(si[file_idx])   
            X[start:end] = si 
        
            # Unnecessary for autoencoder
            # Read each csv to get the labels
            #file_labels = pd.read_csv(self.labels[i])["Population"].to_numpy()
            #y[start:end] = file_labels[file_idx]
        
            # Update the indexing
            start = end
            idx = idx[idx >= max_file_index]
            prev_max = max_file_index
            
        return X, X
    
    def on_epoch_end(self):
        
        if self.shuffle:
            np.random.shuffle(self.indexes)
        
        
    

In [32]:
# Split files for training, validation, testing
#DG = DataGenerator(FCS_data, labels)
TrainDG = DataGenerator(FCS_data[:6], labels[:6])
ValidateDG = DataGenerator([FCS_data[7]], [labels[7]])
TestDG = DataGenerator([FCS_data[8]], [labels[8]])

In [33]:
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

In [34]:
latent_dim = 2

encoder_inputs = keras.Input(shape=(51, 51, 1))
x = layers.Conv2D(64, 3, activation="relu", strides=2, padding="same")(encoder_inputs)
x = layers.Conv2D(128, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Flatten()(x)
x = layers.Dense(16, activation="relu")(x)
z_mean = layers.Dense(51, name="z_mean")(x)
z_log_var = layers.Dense(51, name="z_log_var")(x)
z = Sampling()([z_mean, z_log_var])
encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
encoder.summary()

Model: "encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 51, 51, 1)]  0                                            
__________________________________________________________________________________________________
conv2d_4 (Conv2D)               (None, 26, 26, 64)   640         input_5[0][0]                    
__________________________________________________________________________________________________
conv2d_5 (Conv2D)               (None, 13, 13, 128)  73856       conv2d_4[0][0]                   
__________________________________________________________________________________________________
flatten_2 (Flatten)             (None, 21632)        0           conv2d_5[0][0]                   
____________________________________________________________________________________________

In [35]:
latent_inputs = keras.Input(shape=(latent_dim,))
x = layers.Dense(13 * 13 * 128, activation="relu")(latent_inputs)
x = layers.Reshape((13, 13, 128))(x)
x = layers.Conv2DTranspose(128, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Conv2DTranspose(64, 3, activation="relu", strides=2, padding="same")(x)
decoder_outputs = layers.Conv2DTranspose(1, 3, activation="sigmoid", padding="same")(x)
decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")
decoder.summary()

Model: "decoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 2)]               0         
_________________________________________________________________
dense_5 (Dense)              (None, 21632)             64896     
_________________________________________________________________
reshape_2 (Reshape)          (None, 13, 13, 128)       0         
_________________________________________________________________
conv2d_transpose_6 (Conv2DTr (None, 26, 26, 128)       147584    
_________________________________________________________________
conv2d_transpose_7 (Conv2DTr (None, 52, 52, 64)        73792     
_________________________________________________________________
conv2d_transpose_8 (Conv2DTr (None, 52, 52, 1)         577       
Total params: 286,849
Trainable params: 286,849
Non-trainable params: 0
_____________________________________________________

In [42]:
class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            reconstruction_loss = tf.reduce_mean(
                tf.reduce_sum(
                    keras.losses.binary_crossentropy(data, reconstruction), axis=(1, 2)
                )
            )
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

In [44]:
vae = VAE(encoder, decoder)
vae.compile(optimizer=keras.optimizers.Adam())
history_vae = vae.fit(x=TrainDG,epochs=30, batch_size=128)

NotImplementedError: When subclassing the `Model` class, you should implement a `call` method.

In [16]:
FCS_data[0]

'SamusikData/BM2_cct_normalized_01_non-Neutrophils.fcs'

In [17]:
s0 = FlowCal.io.FCSData(FCS_data[0])
s0 = np.arcsinh(s0) 

In [18]:
s0

FCSData([[ 7.042287 ,  3.7847059,  6.125353 , ...,  5.883358 ,
           1.7685052,  4.0725865],
         [ 7.446585 ,  4.159127 ,  5.8767047, ...,  5.5326867,
           2.752151 ,  4.169027 ],
         [ 8.724532 ,  3.9893267,  4.7816234, ...,  4.845959 ,
           2.514295 ,  4.2849665],
         ...,
         [17.445404 ,  3.9893267,  6.055344 , ...,  5.674298 ,
           1.7874815,  4.2395387],
         [17.445557 ,  3.5842896,  5.6218777, ...,  5.1606765,
           2.031187 ,  4.198265 ],
         [17.446049 ,  3.8291137,  6.3780284, ...,  4.9677305,
          -0.4812681,  4.2320766]], dtype=float32)

In [22]:
pd.DataFrame(s0)

0         1         2         3         4         5         6   \
0       7.042287  3.784706  6.125353  5.494129  6.056492  3.214027  2.020824   
1       7.446585  4.159127  5.876705  5.715124  6.000936  2.874081  3.244338   
2       8.724532  3.989327  4.781623  4.855300  5.139476  2.913752  2.562167   
3       8.726481  3.584290  5.666942  5.600284  5.703247  1.219747  0.852185   
4       9.418329  3.871635  6.183575  5.473680  6.182304  2.982152  2.269284   
...          ...       ...       ...       ...       ...       ...       ...   
86859  17.445086  3.989327  6.690465  6.541294  6.674158  3.385080  3.224220   
86860  17.445097  3.689504  5.658623  5.363382  5.837199  2.895889  2.456636   
86861  17.445404  3.989327  6.055344  5.969394  6.162300  2.445683  2.241683   
86862  17.445557  3.584290  5.621878  5.885203  5.885273  3.468109  1.020490   
86863  17.446049  3.829114  6.378028  5.677200  6.127383  2.786384  1.161602   

             7         8         9   ...        41        42        43  \
0      0.897831 -0.520515  2.958994  ...  2.230268  0.488820 -0.156143   
1      1.555175 -0.640372  1.432310  ...  1.897213  0.962995 -0.519505   
2      2.044967 -0.292645  3.382575  ...  1.842057 -0.579005 -0.582109   
3      0.341788 -0.526864  3.394175  ... -0.662950  0.047379  0.306836   
4      1.594040 -0.692336  3.656763  ...  4.398457 -0.206905  0.794357   
...         ...       ...       ...  ...       ...       ...       ...   
86859  2.368057 -0.196326  3.033409  ...  0.972578 -0.047270 -0.778359   
86860 -0.671567 -0.034089  3.550360  ...  3.602382 -0.284080 -0.397501   
86861  2.381724 -0.790381  2.273468  ...  1.679463  2.551750 -0.624429   
86862  2.428012 -0.080302  4.258890  ... -0.128926  0.896880 -0.056661   
86863  2.371198 -0.528159  2.994852  ...  3.095436 -0.540488 -0.634575   

             44        45        46        47        48        49        50  
0     -0.243717  4.968301  0.689945  4.899695  5.883358  1.768505  4.072587  
1      1.453211  5.614215  1.738823  4.964808  5.532687  2.752151  4.169027  
2      0.162561  6.131474  4.635990  4.136228  4.845959  2.514295  4.284966  
3      6.720085  1.825262  5.285323  4.913344  5.878141  0.470366  3.562895  
4      0.755058  6.099041  2.273190  5.412904  5.577255  1.985156  4.249860  
...         ...       ...       ...       ...       ...       ...       ...  
86859  1.677319  6.381682  2.253848  5.182476  5.949152  2.313524  4.245450  
86860 -0.480340  2.753675  0.717635  4.011479  5.588458  1.909419  3.773996  
86861  1.754761  6.164911  1.394065  5.232368  5.674298  1.787482  4.239539  
86862  1.368072  6.681786  2.613123  4.966264  5.160676  2.031187  4.198265  
86863 -0.362852  5.797256  1.918741  4.197508  4.967731 -0.481268  4.232077  

[86864 rows x 51 columns]

In [45]:
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

In [68]:
latent_dim = 2

encoder_inputs = keras.Input(shape=(None,51))
encoder = Sequential()
encoder.add(layers.Conv1D(32, 3, activation="relu", strides=1, padding="same"))
encoder.add(layers.Conv1D(64, 3, activation="relu", strides=1, padding="same"))
encoder.add(layers.Flatten())
encoder.add(layers.Dense(10, activation="relu"))
# encoder.add(layers.Dense(latent_dim, name="z_mean"))
# encoder.add(layers.Dense(latent_dim, name="z_log_var"))
encoder.compile(loss="mse", optimizer="adam", metrics=["mse"])
history_encoder = encoder.fit(x=TrainDG, 
                              use_multiprocessing=True,
                              workers=-1,
                              epochs=50,
                              validation_data=ValidateDG)
encoder.summary()

ValueError: Input 0 of layer sequential_7 is incompatible with the layer: : expected min_ndim=3, found ndim=2. Full shape received: (30000, 51)

In [47]:
latent_inputs = keras.Input(shape=(latent_dim,))
x = layers.Dense(7 * 7 * 64, activation="relu")(latent_inputs)
x = layers.Reshape((7, 7, 64))(x)
x = layers.Conv2DTranspose(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Conv2DTranspose(32, 3, activation="relu", strides=2, padding="same")(x)
decoder_outputs = layers.Conv2DTranspose(1, 3, activation="sigmoid", padding="same")(x)
decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")
decoder.summary()

Model: "decoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, 2)]               0         
_________________________________________________________________
dense_7 (Dense)              (None, 3136)              9408      
_________________________________________________________________
reshape_3 (Reshape)          (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_transpose_9 (Conv2DTr (None, 14, 14, 64)        36928     
_________________________________________________________________
conv2d_transpose_10 (Conv2DT (None, 28, 28, 32)        18464     
_________________________________________________________________
conv2d_transpose_11 (Conv2DT (None, 28, 28, 1)         289       
Total params: 65,089
Trainable params: 65,089
Non-trainable params: 0
_______________________________________________________

In [48]:
class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            reconstruction_loss = tf.reduce_mean(
                tf.reduce_sum(
                    keras.losses.binary_crossentropy(data, reconstruction), axis=(1, 2)
                )
            )
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

In [54]:
(x_train, _), (x_test, _) = keras.datasets.mnist.load_data()
mnist_digits = np.concatenate([x_train, x_test], axis=0)
mnist_digits = np.expand_dims(mnist_digits, -1).astype("float32") / 255

vae = VAE(encoder, decoder)
vae.compile(optimizer=keras.optimizers.Adam())
vae.fit(s0, epochs=30, batch_size=128)

Epoch 1/30


ValueError: in user code:

    C:\Users\robby\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:805 train_function  *
        return step_function(self, iterator)
    C:\Users\robby\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\robby\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\robby\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\robby\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\robby\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:788 run_step  **
 

In [51]:
print(x_train)

[[[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 ...

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]]
